In [6]:
%pip install pandas openpyxl
%pip install python-docx

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/253.0 kB ? eta -:--:--
   ---------------------------------------- 0.0/253.0 kB ? eta -:--:--
   - -------------------------------------- 10.2/253.0 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/253.0 kB 435.7 kB/s eta 0:00:01
   --------- ----------------------------- 61.4/253.0 kB 550.5 kB/s eta 0:00:01
   -------------------------------- ------- 204.8/253.0 kB 1.2 MB/s eta 0:00:01
   ---------------------------------------- 253.0/253.0 kB 1.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import os

from google import genai
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('API_KEY')

client = genai.Client(api_key=api_key)


In [2]:
import logging

logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

def get_logs(func):
    def wrapper(*args, **kwargs):
        logging.info(f"Executing {func.__name__}")
        data = func(*args, **kwargs)
        return data
    return wrapper


In [ ]:
from docx import Document

@get_logs
def read_document(file_path):
    """
    Read content from a Word document (.docx file)
    
    Args:
        file_path (str): Path to the .docx file
    
    Returns:
        str: Extracted text content from the document
    """
    """
    Reads the text content from a DOCX file.

    Args:
        file_path (str): The path to the DOCX file.

    Returns:
        str: The concatenated text content of the document.
    """
    try:
        document = Document(file_path)
        full_text = []
        for paragraph in document.paragraphs:
            full_text.append(paragraph.text)
        return '\n'.join(full_text)
    except Exception as e:
        return f"Error reading DOCX file: {e}"

In [14]:
prev_response = None

query = 'C:/Users/Varun_Singhal/Downloads/python_interview.docx'
current_query = None

system_prompt = """You are an summarizing agent, which takes in the word document containing interview querstions and gives out a list of 5 most important questions for interview.
Based on the input, provide an output in one of the following format:
1. read_document(directory_url)
2. Final Result: <important questions list>

Answer in one of the above options, do not include multiple responses. Do not include new line characters at the end of the response.

Produt URL:
"""

for i in range(3):
    if not prev_response:
        current_query = f"{system_prompt}{query}\n\n"
    else:
        prompt = f"""In the previous interatction, you responsed with {prev_response}. Here is an additional context {query}. What should be done next ?\n\n"""
        current_query = f"{current_query}{prompt}\n\n"

    prev_response = response = client.models.generate_content(
    model='gemini-2.0-flash',
    contents=current_query
    )
    prev_response = prev_response.text

    logging.info(f"Iteration {str(i+1)}: {prev_response}")

    if prev_response.startswith("Final Result"):
        break

    elif prev_response.startswith("read_document"):
        query = read_document(query)

2025-10-09 09:57:48 - INFO - AFC is enabled with max remote calls: 10.
2025-10-09 09:57:49 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-10-09 09:57:49 - INFO - Iteration 1: read_document(C:/Users/Varun_Singhal/Downloads/python_interview.docx)

2025-10-09 09:57:49 - INFO - Executing read_document
2025-10-09 09:57:49 - INFO - AFC is enabled with max remote calls: 10.
2025-10-09 09:57:50 - INFO - HTTP Request: POST https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent "HTTP/1.1 200 OK"
2025-10-09 09:57:50 - INFO - Iteration 2: Final Result: ["How does Python's garbage collection work?", "What is MRO (Method Resolution Order), method resolution order?", "What is closure in python?", "How is dictionary implemented in Python (talk about key collision etc.)", "Multithreading vs multiprocessing. When to use which one?"]

